In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('r')
sc = SparkContext(conf = conf)

24/08/05 22:16:54 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 192.168.0.51 instead (on interface en0)
24/08/05 22:16:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/05 22:16:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 계좌번호, 거래금액
acTransList = ["SB10001,1000", "SB10002,1200", "SB10003,8000", "SB10004,400", "SB10005,300", "SB10006,10000", "SB10007,500", "SB10008,56", "SB10009,30","SB10010,7000", "CR10001,7000", "SB10002,-10"]
acTransRDD = sc.parallelize(acTransList)
acTransRDD.collect()

['SB10001,1000',
 'SB10002,1200',
 'SB10003,8000',
 'SB10004,400',
 'SB10005,300',
 'SB10006,10000',
 'SB10007,500',
 'SB10008,56',
 'SB10009,30',
 'SB10010,7000',
 'CR10001,7000',
 'SB10002,-10']

In [4]:
# 거래 금액이 0원 이상이고, 계좌번호가 SB로시작하는 데이터
from decimal import Decimal
goodTransRecords = acTransRDD.filter(lambda trans : Decimal(trans.split(',')[1]) > 0).filter(lambda trans : trans.split(',')[0].startswith('SB') == True)
goodTransRecords.collect()

['SB10001,1000',
 'SB10002,1200',
 'SB10003,8000',
 'SB10004,400',
 'SB10005,300',
 'SB10006,10000',
 'SB10007,500',
 'SB10008,56',
 'SB10009,30',
 'SB10010,7000']

In [5]:
# 거래 금액이 1000이상인 모든 고객
upper_1000_Recoard = acTransRDD.filter(lambda trans : Decimal(trans.split(',')[1]) >= 1000)
upper_1000_Recoard.collect()

['SB10001,1000',
 'SB10002,1200',
 'SB10003,8000',
 'SB10006,10000',
 'SB10010,7000',
 'CR10001,7000']

In [12]:
# 계좌번호 'SB'로 시작하지 않는 계좌만 출력
badAcNoLambda = acTransRDD.filter(lambda trans : trans.split(',')[0].startswith('SB') == False)
badAcNoLambda.collect()

['CR10001,7000']

In [23]:
# 모든 거래 금액의 합계
from decimal import Decimal
sumAmount = acTransRDD.map(lambda trans : Decimal(trans.split(',')[1])).reduce(lambda x,y : x+y)
sumAmount

Decimal('35476')

In [29]:
# 모든 거래 금액의 최댓값
maxAmount = acTransRDD.map(lambda trans : Decimal(trans.split(',')[1])).reduce(lambda x, y : x if x>y else y)
maxAmount

Decimal('10000')

In [30]:
# 모든 거래 금액의 최솟값
minAmount = acTransRDD.map(lambda trans : Decimal(trans.split(',')[1])).reduce(lambda x,y : x if x < y else y)
minAmount

Decimal('-10')

In [37]:
# 'SB'로 시작하지 않는 계좌만 출력
allGoodAccountNos = acTransRDD.flatMap(lambda trans : trans.split(',')).filter(lambda trans : trans.startswith('SB')==True)
allGoodAccountNos.collect()

['SB10001',
 'SB10002',
 'SB10003',
 'SB10004',
 'SB10005',
 'SB10006',
 'SB10007',
 'SB10008',
 'SB10009',
 'SB10010',
 'SB10002']